In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf

# Load Data

In [14]:
def load_images(image_folder, image_size=(64, 64)):
    images = []
    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        
        if img is not None:   
            img = cv2.resize(img, image_size)  # Resize to a standard size
            img = (img / 127.5) - 1  # Normalize to [-1, 1]
            images.append(img.astype(np.float32))
        
    return np.array(images)


real_images = load_images("C:\\Users\\Deepak kumar\\GAN_Project\\Dataset\\casia_v2\\Au_jpg")
fake_images = load_images("C:\\Users\\Deepak kumar\\GAN_Project\\Dataset\\casia_v2\\Tp_jpg")

In [20]:
train_real = real_images[:int(0.8 * len(real_images))]
train_fake = fake_images[:int(0.8 * len(fake_images))]
test_real = real_images[int(0.8 * len(real_images)):]
test_fake = fake_images[int(0.8 * len(fake_images)):]

print(f"Training real images: {len(train_real)}, Training fake images: {len(train_fake)}")
print(f"Testing real images: {len(test_real)}, Testing fake images: {len(test_fake)}")

Training real images: 5949, Training fake images: 4098
Testing real images: 1488, Testing fake images: 1025


In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

num_augmented_needed = len(real_images) - len(fake_images)

for img in fake_images:
    img = cv2.resize(img, (64, 64))  
    img = (img / 127.5) - 1 
    img = img.reshape((1,) + img.shape)
    
    # Generate augmented images and save them
    i = 0
    for batch in datagen.flow(img, batch_size=1):
        cv2.imwrite(os.path.join("C:\\Users\\Deepak kumar\\GAN_Project\\Dataset\\casia_v2\\Tp_jpg", f'augmented_fake_{i}.jpg'), batch[0])
        i += 1
        if i >= num_augmented_needed // len(fake_images):
            break

print(f"Generated {num_augmented_needed} augmented fake images.")

Generated 2314 augmented fake images.


# Load Models

In [16]:
from tensorflow.keras.models import load_model

# Load the models
generator_A_to_B = load_model("C:\\Users\\Deepak kumar\\GAN_Project\\Trained_model\\discriminator_A.h5")
generator_B_to_A = load_model("C:\\Users\\Deepak kumar\\GAN_Project\\Trained_model\\generator_B_to_A.h5")
discriminator_A = load_model("C:\\Users\\Deepak kumar\\GAN_Project\\Trained_model\\discriminator_A.h5")
discriminator_B = load_model("C:\\Users\\Deepak kumar\\GAN_Project\\Trained_model\\discriminator_B.h5")

print("Models loaded successfully.")

Models loaded successfully.


In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5) 

# Compile discriminators
discriminator_A.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
discriminator_B.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Classifying Real vs Fake Image

In [18]:
def classify_image(image, discriminator):
    prediction = discriminator(image).numpy().mean()
    return "Real" if prediction >= 0.5 else "Fake"

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Lists to store true labels and predictions
true_labels = []
predictions = []

# Classify real images in the test set
for img in test_real:
    img_resized = cv2.resize(img, (64, 64))
    img_normalized = (img_resized / 127.5) - 1
    pred = classify_image(img_normalized[np.newaxis, ...], discriminator_A)
    predictions.append(1 if pred == "Real" else 0)
    true_labels.append(1)  # True label for real images is 1

# Classify fake images in the test set
for img in test_fake:
    img_resized = cv2.resize(img, (64, 64))
    img_normalized = (img_resized / 127.5) - 1
    pred = classify_image(img_normalized[np.newaxis, ...], discriminator_A)
    predictions.append(1 if pred == "Real" else 0)
    true_labels.append(0)  # True label for fake images is 0

# Step 2: Calculate Performance Metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Discriminator Performance on Test Set:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")


Discriminator Performance on Test Set:
Accuracy: 59.21%
Precision: 59.21%
Recall: 100.00%
F1 Score: 74.38%


In [5]:
# # Test with other Image

In [13]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))  
    img = (img / 127.5) - 1 
    return np.expand_dims(img, axis=0)  

test_image_path = "C:\\Users\\Deepak kumar\\GAN_Project\\Random_Images\\girl1.jpg"  
test_image = preprocess_image(test_image_path)

logits = discriminator_A.predict(test_image)  
prediction = tf.sigmoid(logits)
prediction = tf.reduce_mean(prediction).numpy()
result = "Real" if prediction >= 0.5 else "Fake"
print(f"The image is classified as: {result} !!!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
The image is classified as: 0.681841254234314 !!!
